### figure S1i

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import adata as ad
import scanpy as sc

In [ ]:
sample_list = pd.read_csv("HD-OV 100.csv")

In [ ]:
adata_list = []

for index, row in sample_list.iterrows():
    sample = row['sample_id']
    file_path = row['path']
    subtype = row['subtype']

    adata = sc.read_h5ad(f"{file_path}/outs/adata_8um.h5ad")
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
    adata.obs['array_row'] = adata.obs['array_row'].astype(float)
    adata.obs['array_col'] = adata.obs['array_col'].astype(float)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)

    adata_list.append(adata)


In [ ]:
adata_merged = ad.concat(adata_list, axis=0, merge='same', pairwise=True)
adata_merged.var["mt"] = adata_merged.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(
    adata_merged, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)
obs = adata_merged.obs


In [ ]:
palette = {
        'NK': '#CDCE6B',
        'T':'#FFFF99',
        'B':'#00CC99',
        'DC':'#FF9999',
        'Mast':'#6699FF',
        'Neutrophil':'#3366CC',
        'Macrophage':'#990066',
        'Endothelial': '#FF3366',
        'Fibroblast': '#008080',
        'Malignant': '#0D5886',
        'Others': 'lightgray'
}

order = ['NK', 'T', 'B', 'DC', 'Mast', 'Neutrophil', 'Macrophage', 'Endothelial',  'Fibroblast', 'Malignant', 'Others']
obs['annotations'] = pd.Categorical(obs['annotations'], categories=order, ordered=True)


In [ ]:
sns.violinplot(x='annotations', y='pct_counts_mt', data=obs, palette=palette, inner=None, order=order)

plt.xlabel('Cell Type')
plt.ylabel('Percent mtDNA')
plt.xticks(rotation=30, ha='right')
plt.show()

In [ ]:
sns.violinplot(x='annotations', y='n_genes', data=obs, palette=palette, inner=None, order=order)

plt.xlabel('Cell Type')
plt.ylabel('nFeature_RNA')
plt.xticks(rotation=30, ha='right')
plt.show()

In [ ]:
sns.violinplot(x='annotations', y='total_counts', data=obs, palette=palette, inner=None, order=order)

plt.xlabel('Cell Type')
plt.ylabel('nCount_RNA')
plt.xticks(rotation=30, ha='right')
plt.show()
